In [2]:
#import
import os
import pandas as pd
import datetime

import pymongo
from pymongo import MongoClient 

In [3]:
# CVS files into dataframes
city_df =pd.read_csv(os.path.join('data/city_attributes_US.csv'))
speed_df =pd.read_csv(os.path.join('data/wind_speed_US.csv'))
direction_df =pd.read_csv(os.path.join('data/wind_direction_US.csv'))

In [4]:
city_df = city_df.drop(['Country'], axis=1)

In [5]:
city_df = city_df.set_index('City')

city_df.head()

#city_df.info()

,Latitude,Longitude
City,,
Portland,45.523449,-122.676208
San Francisco,37.774929,-122.419418
Seattle,47.606209,-122.332069
Los Angeles,34.052231,-118.243683
San Diego,32.715328,-117.157257


In [7]:
speed_df =speed_df.dropna()  #drop ~1.8% of rows, drop rows where at least one element is missing.

In [8]:
direction_df =direction_df.dropna()  #drop ~1.8% of rows, drop rows where at least one element is missing.

In [9]:
direction_df.iloc[:,1:28] = direction_df.iloc[:,1:28].astype(float)
direction_df['datetime'] = pd.to_datetime(direction_df['datetime'])

In [10]:
speed_df.iloc[:,1:28] = speed_df.iloc[:,1:28].astype(float)
speed_df['datetime'] = pd.to_datetime(speed_df['datetime'])

In [11]:
speed_df['month'] =speed_df['datetime'].dt.month
speed_df['year'] =speed_df['datetime'].dt.year
ave_speed_df = speed_df.groupby(['year', 'month']).mean()

direction_df['month'] =direction_df['datetime'].dt.month
direction_df['year'] =direction_df['datetime'].dt.year
ave_direction_df = direction_df.groupby(['year', 'month']).mean()

In [23]:
tave_speed =ave_speed_df.transpose().to_dict('index')
tave_direction =ave_direction_df.transpose().to_dict('index')
latlon =city_df.to_dict('index')

# Continueing Data Clean Up to find the average change in mph and direction

In [17]:
ave_speed_df.head()

Portland  San Francisco   Seattle  Los Angeles  San Diego  \
year month                                                              
2012 10     2.485636       3.206566  2.280438     0.592339   1.742818   
     11     2.608333       2.597222  1.870833     0.644444   1.755556   
     12     3.219086       3.373656  2.701613     0.592742   1.697581   
2013 1      2.361559       2.291667  1.512097     0.775538   1.524194   
     2      2.010417       3.266369  2.334821     0.690476   1.760417   

            Las Vegas   Phoenix  Albuquerque    Denver  San Antonio  ...  \
year month                                                           ...   
2012 10      2.359781  1.157319     2.496580  2.692202     2.569083  ...   
     11      1.945833  0.779167     1.750000  2.306944     1.856944  ...   
     12      2.533602  0.931452     2.158602  2.768817     2.348118  ...   
2013 1       2.463710  1.258065     2.731183  2.338710     2.649194  ...   
     2       2.663690  1.325893     3.526786  3.223214     3.166667  ...   

            Indianapolis   Atlanta   Detroit  Jacksonville  Charlotte  \
year month                                                              
2012 10         4.112175  1.781122  3.087551      3.459644   2.065663   
     11         2.880556  1.641667  2.475000      3.172222   2.262500   
     12         4.016129  1.880376  2.704301      2.834677   2.241935   
2013 1          4.088710  1.883065  3.428763      3.220430   2.272849   
     2          4.535714  2.525298  3.904762      3.373512   2.678571   

               Miami  Pittsburgh  Philadelphia  New York    Boston  
year month                                                          
2012 10     3.225718    2.760602      3.049248  4.094391  2.595075  
     11     2.713889    2.519444      3.241667  4.041667  2.477778  
     12     2.694892    3.125000      3.227151  4.509409  2.196237  
2013 1      3.196237    3.354839      3.159946  4.436828  2.876344  
     2      3.272321    3.635417      4.279762  5.151786  3.489583  

[5 rows x 27 columns]

In [19]:
avg_change_mph = ave_speed_df - ave_speed_df.shift()
avg_change_mph.head()

Portland  San Francisco   Seattle  Los Angeles  San Diego  \
year month                                                              
2012 10          NaN            NaN       NaN          NaN        NaN   
     11     0.122697      -0.609344 -0.409604     0.052105   0.012737   
     12     0.610753       0.776434  0.830780    -0.051703  -0.057975   
2013 1     -0.857527      -1.081989 -1.189516     0.182796  -0.173387   
     2     -0.351142       0.974702  0.822725    -0.085061   0.236223   

            Las Vegas   Phoenix  Albuquerque    Denver  San Antonio  ...  \
year month                                                           ...   
2012 10           NaN       NaN          NaN       NaN          NaN  ...   
     11     -0.413948 -0.378152    -0.746580 -0.385258    -0.712139  ...   
     12      0.587769  0.152285     0.408602  0.461873     0.491174  ...   
2013 1      -0.069892  0.326613     0.572581 -0.430108     0.301075  ...   
     2       0.199981  0.067828     0.795603  0.884505     0.517473  ...   

            Indianapolis   Atlanta   Detroit  Jacksonville  Charlotte  \
year month                                                              
2012 10              NaN       NaN       NaN           NaN        NaN   
     11        -1.231620 -0.139455 -0.612551     -0.287422   0.196837   
     12         1.135573  0.238710  0.229301     -0.337545  -0.020565   
2013 1          0.072581  0.002688  0.724462      0.385753   0.030914   
     2          0.447005  0.642233  0.475998      0.153082   0.405722   

               Miami  Pittsburgh  Philadelphia  New York    Boston  
year month                                                          
2012 10          NaN         NaN           NaN       NaN       NaN  
     11    -0.511829   -0.241157      0.192419 -0.052725 -0.117297  
     12    -0.018996    0.605556     -0.014516  0.467742 -0.281541  
2013 1      0.501344    0.229839     -0.067204 -0.072581  0.680108  
     2      0.076085    0.280578      1.119816  0.714958  0.613239  

[5 rows x 27 columns]

In [22]:
# drop first row because of NaN values
avg_change_mph = avg_change_mph.drop(avg_change_mph.index[0])
avg_change_mph.head()

Portland  San Francisco   Seattle  Los Angeles  San Diego  \
year month                                                              
2012 11     0.122697      -0.609344 -0.409604     0.052105   0.012737   
     12     0.610753       0.776434  0.830780    -0.051703  -0.057975   
2013 1     -0.857527      -1.081989 -1.189516     0.182796  -0.173387   
     2     -0.351142       0.974702  0.822725    -0.085061   0.236223   
     3     -0.619288      -0.117175 -0.447725     0.031298  -0.097782   

            Las Vegas   Phoenix  Albuquerque    Denver  San Antonio  ...  \
year month                                                           ...   
2012 11     -0.413948 -0.378152    -0.746580 -0.385258    -0.712139  ...   
     12      0.587769  0.152285     0.408602  0.461873     0.491174  ...   
2013 1      -0.069892  0.326613     0.572581 -0.430108     0.301075  ...   
     2       0.199981  0.067828     0.795603  0.884505     0.517473  ...   
     3      -0.081701 -0.725086    -0.698829 -0.721870     0.044355  ...   

            Indianapolis   Atlanta   Detroit  Jacksonville  Charlotte  \
year month                                                              
2012 11        -1.231620 -0.139455 -0.612551     -0.287422   0.196837   
     12         1.135573  0.238710  0.229301     -0.337545  -0.020565   
2013 1          0.072581  0.002688  0.724462      0.385753   0.030914   
     2          0.447005  0.642233  0.475998      0.153082   0.405722   
     3         -0.541091  0.500240 -0.517665     -0.228351  -0.433948   

               Miami  Pittsburgh  Philadelphia  New York    Boston  
year month                                                          
2012 11    -0.511829   -0.241157      0.192419 -0.052725 -0.117297  
     12    -0.018996    0.605556     -0.014516  0.467742 -0.281541  
2013 1      0.501344    0.229839     -0.067204 -0.072581  0.680108  
     2      0.076085    0.280578      1.119816  0.714958  0.613239  
     3     -1.168827   -0.376008     -0.846966 -1.431356 -0.359207  

[5 rows x 27 columns]

In [30]:
ave_direction_df.head()

Portland  San Francisco     Seattle  Los Angeles   San Diego  \
year month                                                                   
2012 10     157.555404     189.697674  124.455540    45.761970  163.300958   
     11     130.190278     151.704167  104.461111    56.527778  158.216667   
     12     136.665323     164.856183  124.349462    40.948925  147.934140   
2013 1      127.737903     126.782258  101.099462    58.317204  132.831989   
     2      138.174107     196.654762  128.934524    49.467262  147.110119   

             Las Vegas     Phoenix  Albuquerque      Denver  San Antonio  ...  \
year month                                                                ...   
2012 10     164.139535   91.082079   171.232558  160.723666   124.166895  ...   
     11     172.234722   84.641667   139.737500  160.558333   109.025000  ...   
     12     157.315860   75.290323   152.594086  169.494624   156.028226  ...   
2013 1      181.448925  100.456989   189.448925  159.923387   147.745968  ...   
     2      182.842262  100.757440   220.992560  188.209821   162.104167  ...   

            Indianapolis     Atlanta     Detroit  Jacksonville   Charlotte  \
year month                                                                   
2012 10       193.120383  129.411765  181.742818    159.667579  123.051984   
     11       171.106944  121.168056  177.497222    180.372222  127.231944   
     12       189.634409  116.508065  165.713710    173.204301  147.615591   
2013 1        206.083333  137.184140  210.513441    181.189516  143.911290   
     2        211.555060  154.806548  211.431548    194.724702  159.764881   

                 Miami  Pittsburgh  Philadelphia    New York      Boston  
year month                                                                
2012 10     132.155951  191.222982    161.473324  163.690834  126.786594  
     11     149.194444  186.863889    187.926389  185.325000  150.455556  
     12     128.604839  188.022849    170.793011  179.750000  145.283602  
2013 1      114.146505  208.506720    225.693548  234.081989  193.561828  
     2      149.498512  205.616071    203.507440  205.383929  180.683036  

[5 rows x 27 columns]

In [25]:
avg_change_dir = ave_direction_df - ave_direction_df.shift()
avg_change_dir.head()

Portland  San Francisco    Seattle  Los Angeles  San Diego  \
year month                                                                
2012 10           NaN            NaN        NaN          NaN        NaN   
     11    -27.365126     -37.993508 -19.994429    10.765808  -5.084291   
     12      6.475045      13.152016  19.888351   -15.578853 -10.282527   
2013 1      -8.927419     -38.073925 -23.250000    17.368280 -15.102151   
     2      10.436204      69.872504  27.835061    -8.849942  14.278130   

            Las Vegas    Phoenix  Albuquerque     Denver  San Antonio  ...  \
year month                                                             ...   
2012 10           NaN        NaN          NaN        NaN          NaN  ...   
     11      8.095187  -6.440413   -31.495058  -0.165333   -15.141895  ...   
     12    -14.918862  -9.351344    12.856586   8.936290    47.003226  ...   
2013 1      24.133065  25.166667    36.854839  -9.571237    -8.282258  ...   
     2       1.393337   0.300451    31.543635  28.286434    14.358199  ...   

            Indianapolis    Atlanta    Detroit  Jacksonville  Charlotte  \
year month                                                                
2012 10              NaN        NaN        NaN           NaN        NaN   
     11       -22.013439  -8.243709  -4.245596     20.704644   4.179961   
     12        18.527464  -4.659991 -11.783513     -7.167921  20.383647   
2013 1         16.448925  20.676075  44.799731      7.985215  -3.704301   
     2          5.471726  17.622408   0.918107     13.535186  15.853591   

                Miami  Pittsburgh  Philadelphia   New York     Boston  
year month                                                             
2012 10           NaN         NaN           NaN        NaN        NaN  
     11     17.038494   -4.359093     26.453065  21.634166  23.668962  
     12    -20.589606    1.158961    -17.133378  -5.575000  -5.171953  
2013 1     -14.458333   20.483871     54.900538  54.331989  48.278226  
     2      35.352007   -2.890649    -22.186108 -28.698061 -12.878792  

[5 rows x 27 columns]

In [26]:
avg_change_dir = avg_change_dir.drop(avg_change_dir.index[0])
avg_change_dir.head()

Portland  San Francisco    Seattle  Los Angeles  San Diego  \
year month                                                                
2012 11    -27.365126     -37.993508 -19.994429    10.765808  -5.084291   
     12      6.475045      13.152016  19.888351   -15.578853 -10.282527   
2013 1      -8.927419     -38.073925 -23.250000    17.368280 -15.102151   
     2      10.436204      69.872504  27.835061    -8.849942  14.278130   
     3      -9.319268      -2.068740   5.242896    43.636233  23.696333   

            Las Vegas    Phoenix  Albuquerque     Denver  San Antonio  ...  \
year month                                                             ...   
2012 11      8.095187  -6.440413   -31.495058  -0.165333   -15.141895  ...   
     12    -14.918862  -9.351344    12.856586   8.936290    47.003226  ...   
2013 1      24.133065  25.166667    36.854839  -9.571237    -8.282258  ...   
     2       1.393337   0.300451    31.543635  28.286434    14.358199  ...   
     3     -24.616455  57.192828   -10.937452   2.126200    -3.452285  ...   

            Indianapolis    Atlanta    Detroit  Jacksonville  Charlotte  \
year month                                                                
2012 11       -22.013439  -8.243709  -4.245596     20.704644   4.179961   
     12        18.527464  -4.659991 -11.783513     -7.167921  20.383647   
2013 1         16.448925  20.676075  44.799731      7.985215  -3.704301   
     2          5.471726  17.622408   0.918107     13.535186  15.853591   
     3         -3.263393  62.939420 -15.254128    -12.794595  15.810388   

                Miami  Pittsburgh  Philadelphia   New York     Boston  
year month                                                             
2012 11     17.038494   -4.359093     26.453065  21.634166  23.668962  
     12    -20.589606    1.158961    -17.133378  -5.575000  -5.171953  
2013 1     -14.458333   20.483871     54.900538  54.331989  48.278226  
     2      35.352007   -2.890649    -22.186108 -28.698061 -12.878792  
     3      42.494768    2.104359     17.941484  24.476286  20.096534  

[5 rows x 27 columns]

In [37]:
tavg_change_mph =avg_change_mph.transpose().to_dict('index')
tave_change_direction =avg_change_dir.transpose().to_dict('index')

In [38]:
cities =[]
for city in tave_speed:
    city_name =city
    mph =tave_speed[city].values()
    deg =tave_direction[city].values()
    loc =latlon[city].values()
    mph_change = tavg_change_mph[city].values()
    deg_change = tave_change_direction[city].values()
        
    cities.append({'city':city_name, 
                  'loc': list(loc),
                  'mph': list(mph),
                  'deg': list(deg),
                  'mph_change':list(mph_change),
                  'deg_change':list(deg_change)})
    
    
   

In [39]:
#---------------------------------- Mongo DB ----------------------------------
conn = 'mongodb://localhost:27017/'
client =pymongo.MongoClient(conn)

db = client.smallWind
db.mpg_ang.insert_many(cities)


In [ ]:
# #save to csv files
# city_df.to_csv(os.path.join( 'data/city_clean.csv' ), index=False)
# # speed_df.to_csv(os.path.join( 'data/speed_clean.csv' ), index=False)
# # direction_df.to_csv(os.path.join( 'data/direction_clean.csv' ), index=False)

# speed_direction_df.to_csv(os.path.join( 'data/speed_direction.csv' ), index=False)

# speed_direction_df.to_csv(os.path.join( 'data/speed_direction.csv' ), index=False)